In [19]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import random
import pandas as pd
from time import sleep

In [20]:
url_to_parce = 'https://www.mk.ru/news/'
news_urls = set() # all urls to links
news_lenta = [] # data for all news

In [21]:
def analyze_page(url0):
    """
    Returns a tuple with 
    url0, time_and_date, tags, title_text, full_text
    Parameters:
    url0 is a link to the news (string)
    """

    try:
        page0 = requests.get(url0)
    except:
        return None, None, None, None, None
    
        
    soup0 = BeautifulSoup(page0.text, 'html')

    time_tag  = soup0.find('time', class_ = 'meta__text')             
    try:    
        time_and_date = time_tag.get_text(strip=True)
    except:
        time_and_date = None
    
    tag_elements = soup0.find_all('a', class_='article__tag-item')
    try:
        tags = [tag.get_text() for tag in tag_elements]
    except:
        tags = None
        
    title = (soup0.find('h1', class_ =  'article__title'))
    try:
        title_text = title.text
    except:
        title_text = None

    text_contents = (soup0.find('div', class_ =  'article__body'))
    full_text = text_contents.text

    return url0, time_and_date, tags, title_text, full_text

In [ ]:
for year in tqdm(range(1998, 2023)):
    for month in range(1, 12):
        for day in range(1, 31):

            url = url_to_parce + str(year)+ '/' + str(month) + '/' + str(day) + '/'
                
            try:
                page = requests.get(url)
            except:
                continue
            if page.status_code == 429: #Received a 429 Too Many Requests status.
                time.sleep(500) # Waiting before retrying...
                continue
                    
            soup = BeautifulSoup(page.text, 'html.parser')
            for link in soup.find_all('a', class_='news-listing__item-link'):
                news_urls.add(link.get('href'))

            sleep(5)

 44%|████████████████▎                    | 11/25 [5:41:35<7:16:00, 1868.63s/it]

In [5]:
print(news_urls)

set()


In [26]:
len(news_urls)

df1 = pd.DataFrame(news_urls)
df1.to_excel('urls_mkru_archive.xlsx')

In [ ]:
for link in tqdm(news_urls):
    try:
        res = analyze_page(link)
    except:
        continue
    if res.status_code == 429:
        #print("Received a 429 Too Many Requests status. Waiting before retrying...")
        time.sleep(500)
        res = analyze_page(link)
    # print(res)
    news_lenta.append(res)
    sleep(5)


In [ ]:
df = pd.DataFrame(news_lenta)
df.columns = ['url','date and time', 'tag', 'title', 'text']
df.to_excel('parsed_news_mkru_archive.xlsx')